# Requerimientos del canal
### Instituto Tecnologico de Costa Rica
### Escuela de Ingeniería Electrónica
### Curso: Taller de Comunicaciones Eléctricas
### Estudiantes:
- Esteban Arias Rojas
- David Herrera Castro
- David Monge Naranjo
- Federico Rivera Moya

## Bloque canal

### Paquetes importantes a utilizar

In [1]:
import numpy as np
import pandas as pd
from utils import awgn_channel
import random

### Lectura de los datos codificados

In [2]:
# Datos provenientes de la codificacion con Hamming 1511
file_1511 = "datos_codificados_1511.xlsx"

# Datos provenientes de la codificacion con Hamming 74
file_74   = "datos_codificados_74.xlsx"

# Leer el archivo Excel
codificacion_1511 = pd.read_excel(file_1511)
codificacion_74   = pd.read_excel(file_74)

# Mostrar los datos leídos
print("Datos con codificación Hamming 1511")
print(codificacion_1511)

print("Datos con codificación Hamming 74")
print(codificacion_74)

Datos con codificación Hamming 1511
      Datos
0         0
1         1
2         0
3         1
4         1
...     ...
1390      0
1391      0
1392      0
1393      0
1394      1

[1395 rows x 1 columns]
Datos con codificación Hamming 74
      Datos
0         0
1         1
2         0
3         1
4         1
...     ...
1773      0
1774      0
1775      1
1776      0
1777      1

[1778 rows x 1 columns]


Los datos son un data frame, se requieren pasar a lista para que puedan ser utilizados por la función del canal

In [3]:
datos_1511 = np.array(codificacion_1511)
datos_74   = np.array(codificacion_74)

# Convierte los datos a una lista
datos_list_1511 = []
for bit in datos_1511:
    datos_list_1511.append(datos_1511[bit][0][0])

# Convierte los datos a una lista
datos_list_74 = []
for bit in datos_74:
    datos_list_74.append(datos_74[bit][0][0])

Los datos se encuentran en binario, es necesario pasarlos a decimal ante de agregarles ruido

In [4]:
#Para la codificación Hamming 1511
# Separa los datos en grupos de 15 bits
grupos_de_1511 = [datos_list_1511[i:i+15] for i in range(0, len(datos_list_1511), 15)]

# Convierte cada grupo de bits en una cadena de 15 caracteres
cadenas_1511 = [''.join(map(str, grupo)) for grupo in grupos_de_1511]

# Convierte cada cadena binaria en decimal
numeros_decimales_1511 = [int(cadena, 2) for cadena in cadenas_1511]

#Para la codificación Hamming 74
# Separa los datos en grupos de 7 bits
grupos_de_74 = [datos_list_74[i:i+7] for i in range(0, len(datos_list_74), 7)]

# Convierte cada grupo de bits en una cadena de 7 caracteres
cadenas_74 = [''.join(map(str, grupo)) for grupo in grupos_de_74]

# Convierte cada cadena binaria en decimal
numeros_decimales_74 = [int(cadena, 2) for cadena in cadenas_74]

### Función para pasar los datos por el canal

In [5]:
# Semilla para generar secuencia pseudo aleatoria
seed = 3

# Se llama la funcion que modela el canal
[canal_1511, c_1511] = awgn_channel.apply_awgn_channel(signal=numeros_decimales_1511, snr_db=22, signal_power=5, semilla=seed)
[canal_74, c_74]     = awgn_channel.apply_awgn_channel(signal=numeros_decimales_74, snr_db=22, signal_power=5, semilla=seed)

#Limita los valores que pueden ser representandos por 15 bits desde 0 hasta 32768
canal_1511 = np.clip(canal_1511, 0, 32768)

#Limita los valores que pueden ser representandos por 7 bits desde 0 hasta 127
canal_74 = np.clip(canal_74, 0, 127)

print("Datos después de pasar por el canal:")
print(canal_1511)
print("\nDatos después de pasar por el canal:")
print(canal_74)

Datos después de pasar por el canal:
[11682.31769117 29045.07753165  2989.01713961 24153.66901166
 31484.95073109 22556.93698871 17091.98530369  9784.8886339
 12862.99221714  5530.91523787 19691.76663503 26474.15712414
 17226.15653724 23184.30364957 22225.00888684 14018.92812233
 13875.9031347  13743.72531904 19941.17448535 28532.80443146
   899.78951536  7262.96347304 25327.2639655  10019.04204488
 14774.81815816 20952.87336015 11970.11105426 30579.97149007
 17356.86344145  7764.95914259 29936.1323348  10148.35099125
 15803.77902231 28151.88873759  3970.85723732 31866.57032924
 27640.83591865  8172.81814206   265.19963781  2078.97656976
 16703.7116766   2566.11486068 28823.93672019  1412.69038818
 13389.89402477  8806.89545552  5403.84478348 14542.00527768
 18126.60067077 14898.95244089 21019.1799588   4496.15147156
  5266.19683315 25894.19882303 11557.26421324 26995.80137057
 19534.15023456 26210.66947403 21360.89291724  7325.65995685
 27889.18616902 10210.23689477 16703.96493576 108

Ahora que los datos tienen ruido se decide redondearlos, para después pasarlos a binario y que continúen a la etapa de decodificación

In [6]:
# Se redondean los datos
rounded_data_1511 = [round(number) for number in canal_1511]
rounded_data_74   = [round(number) for number in canal_74]

In [7]:
# Se pasan a un string que contiene binario
binarios_1511 = [bin(numero)[2:].zfill(15) for numero in rounded_data_1511]
binarios_74   = [bin(numero)[2:].zfill(7) for numero in rounded_data_74]

In [8]:
binary_digits_1511 = [int(bit) for binary_string in binarios_1511 for bit in binary_string]
binary_digits_74   = [int(bit) for binary_string in binarios_74 for bit in binary_string]


### Guardar los datos

In [9]:
# Se convierten los datos a un data frame
datos_ruido_1511 = pd.DataFrame({"Datos": binary_digits_1511})
datos_ruido_74   = pd.DataFrame({"Datos": binary_digits_74})

datos_ruido_1511.to_excel("datos_ruido_1511.xlsx", index=False)
datos_ruido_74.to_excel("datos_ruido_74.xlsx", index=False)

### Capacidad del canal

In [10]:
print("Capacidad del canal aplicando Hamming (15,11):", c_1511)
print("Capacidad del canal aplicando Hamming (7,4):", c_74)

Capacidad del canal aplicando Hamming (15,11): 3.6586580009682743
Capacidad del canal aplicando Hamming (7,4): 3.6586580009682743


Los datos acerca de la capacidad del canal que regresa la función del komm no incluye el ancho de banda, por lo que para el sistema utilizado, se quiere operar a B=2.4GHz, por lo que se debe de multiplicar el valor obtenido por 2B

In [11]:
print("Capacidad del canal de Mega bits por segundo:", c_74*2*2.4, "Mb/s")

Capacidad del canal de Mega bits por segundo: 17.561558404647716 Mb/s
